In [6]:
import cv2


PIXEL_TO_CM = 0.0264

def detect_shapes(frame):
    flip = cv2.flip(frame, 1)
    gray = cv2.cvtColor(flip, cv2.COLOR_BGR2GRAY)
    
    blurred = cv2.GaussianBlur(gray, (7, 7), 0)
    edged = cv2.Canny(blurred, 50, 150)
    _, thresh = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
        
        if area > 500:
            if len(approx) == 4:
                x, y, w, h = cv2.boundingRect(approx)
                aspect_ratio = w / float(h)
                if 0.95 <= aspect_ratio <= 1.05:
                    shape_name = "Square"
                else:
                    shape_name = "Rectangle"
                # Calculate length and breadth for rectangle/square in cm
                length_cm = w * PIXEL_TO_CM
                breadth_cm = h * PIXEL_TO_CM
                # Draw contours and shape name
                cv2.drawContours(flip, [approx], -1, (0, 255, 0), 2)
                cv2.putText(flip, f"(L):{length_cm:.2f}cm", (x - 5 + int(w / 2), y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                cv2.putText(flip, f"(B):{breadth_cm:.2f}cm", (x + w, y + int(h / 2)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            elif len(approx) > 4:
                shape_name = "Circle"
                # Calculate diameter for circle in cm
                diameter_cm = (area * 4 / perimeter) * PIXEL_TO_CM  # Diameter = 4 * Area / Perimeter
                # Get the centroid of the circle to place text
                M = cv2.moments(contour)
                if M["m00"] != 0:
                    x = int(M["m10"] / M["m00"])
                    y = int(M["m01"] / M["m00"])
                else:
                    x, y = 0, 0
                # Draw contours and shape name
                cv2.drawContours(flip, [approx], -1, (0, 255, 0), 2)
                cv2.putText(flip, f"D:{diameter_cm:.2f}cm", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return flip, edged

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open video stream from webcam.")
    exit()

while True:
    ret, frame = cap.read()
    
    # If frame is read correctly ret is True
    if not ret:
        print("Error: Cannot receive frame (stream end?). Exiting ...")
        break
    
    # Detect shapes in the frame
    output_frame, edged = detect_shapes(frame)
    
    # Display the resulting frame with shapes
    cv2.imshow('Shape Detection', output_frame)
    # Display the edge-detected frame
    cv2.imshow('Edge Detection', edged)
    
    # Exit the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()
